# League Table

In [1]:
import pandas as pd
import numpy as np

In [2]:
fixtures = pd.read_csv('../data/fpl_official/vaastav/data/2021-22/fixtures.csv')

results = fixtures[fixtures['finished'] == True][['team_a', 'team_a_score', 'team_h', 'team_h_score']]

league_table = pd.read_csv('../data/fpl_official/vaastav/data/2021-22/teams.csv')[['id', 'name', 'short_name']]
league_table['points'] = 0

In [3]:
def score_to_points_at_home(row):
    if row['team_a_score'] == row['team_h_score']:
        return 1
    elif row['team_a_score'] < row['team_h_score']:
        return 3
    else:
        return 0

home_results = results.copy()
home_results['points'] = home_results.apply(lambda row: score_to_points_at_home(row), axis=1)
home_results = (home_results.loc[:, ['team_a', 'team_h', 'points']])

def score_to_points_at_away(row):
    if row['team_a_score'] == row['team_h_score']:
        return 1
    elif row['team_a_score'] > row['team_h_score']:
        return 3
    else:
        return 0

away_results = results.copy()
away_results['points'] = away_results.apply(lambda row: score_to_points_at_away(row), axis=1)
away_results = (away_results.loc[:, ['team_a', 'team_h', 'points']])

In [4]:
def aggregate_points(team):
    return home_results[home_results['team_h'] == team]['points'].sum() +\
           away_results[away_results['team_a'] == team]['points'].sum()

In [5]:
lt = (league_table.id.map(aggregate_points))
league_table['points'] = lt
league_table.sort_values(by=['points'], ascending=False)

,id,name,short_name,points
10,11,Liverpool,LIV,10
5,6,Chelsea,CHE,10
7,8,Everton,EVE,10
12,13,Man Utd,MUN,10
3,4,Brighton,BHA,9
11,12,Man City,MCI,9
16,17,Spurs,TOT,9
18,19,West Ham,WHU,8
8,9,Leicester,LEI,6
2,3,Brentford,BRE,5


# XTable

In [6]:
league_table['xpoints'] = 0

In [7]:
def xg_vs_xga(row):
    if np.round(row['xG'], 2) ==  np.round(row['xGA'], 2):
        return 1
    elif row['xG'] > row['xGA']:
        return 3
    else:
        return 0

In [12]:
league_table.loc[:, 'xpoints'] = [
    (pd.read_csv('../data/understat/2021-22/Arsenal.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Aston_Villa.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Brentford.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Brighton.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Burnley.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Chelsea.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Crystal_Palace.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Everton.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Leeds.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Leicester.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Liverpool.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Manchester_City.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Manchester_United.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Newcastle_United.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Norwich.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Southampton.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Tottenham.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Watford.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/West_Ham.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Wolverhampton_Wanderers.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum())
]
league_table

,id,name,short_name,points,xpoints
0,1,Arsenal,ARS,3,6
1,2,Aston Villa,AVL,4,6
2,3,Brentford,BRE,5,15
3,4,Brighton,BHA,9,6
4,5,Burnley,BUR,1,9
5,6,Chelsea,CHE,10,12
6,7,Crystal Palace,CRY,5,6
7,8,Everton,EVE,10,9
8,9,Leicester,LEI,6,3
9,10,Leeds,LEE,2,0


In [13]:
league_table['npxpoints'] = 0

In [14]:
def xg_vs_xga(row):
    if np.round(row['npxG'], 2) ==  np.round(row['npxGA'], 2):
        return 1
    elif row['npxG'] > row['npxGA']:
        return 3
    else:
        return 0

In [15]:
league_table.loc[:, 'npxpoints'] = [
    (pd.read_csv('../data/understat/2021-22/Arsenal.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Aston_Villa.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Brentford.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Brighton.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Burnley.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Chelsea.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Crystal_Palace.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Everton.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Leeds.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Leicester.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Liverpool.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Manchester_City.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Manchester_United.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Newcastle_United.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Norwich.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Southampton.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Tottenham.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Watford.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/West_Ham.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum()),
    (pd.read_csv('../data/understat/2021-22/Wolverhampton_Wanderers.csv')
     .apply(lambda row: xg_vs_xga(row), axis=1).sum())
]
league_table

,id,name,short_name,points,xpoints,npxpoints
0,1,Arsenal,ARS,3,6,6
1,2,Aston Villa,AVL,4,6,3
2,3,Brentford,BRE,5,15,15
3,4,Brighton,BHA,9,6,3
4,5,Burnley,BUR,1,9,9
5,6,Chelsea,CHE,10,12,12
6,7,Crystal Palace,CRY,5,6,6
7,8,Everton,EVE,10,9,9
8,9,Leicester,LEI,6,3,3
9,10,Leeds,LEE,2,0,6
